# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,puerto baquerizo moreno,-0.9000,-89.6000,72.43,89,28,7.20,EC,1716856807
1,1,pariaman,-0.6190,100.1200,82.45,86,77,2.04,ID,1716856936
2,2,codrington,-38.2667,141.9667,63.21,40,5,14.18,AU,1716856937
3,3,grytviken,-54.2811,-36.5092,30.49,81,24,12.12,GS,1716856939
4,4,mundybash,53.2110,87.2983,51.26,97,100,6.40,RU,1716856940


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [8]:
#%%capture --no-display

# Configure the map plot
humiditycity= city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    size = "Humidity",
    scale = 1,
    color = "City",
    alpha = 0.7,
    tiles = "OSM",
    frame_height = 510,
    frame_width = 900,
    hover_cols = ["City", "Humidity"]
)

# Display the map
humiditycity

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [10]:
# Narrow down cities that fit criteria and drop any results with null values
idealweatherDF = city_data_df.loc[
    (city_data_df["Max Temp"]> 69.8) & (city_data_df["Max Temp"]< 80.6) & (city_data_df["Wind Speed"]< 4.5) & (city_data_df["Cloudiness"]==0)
]

# Drop any rows with null values
idealweatherDF = idealweatherDF.dropna()

# Display sample data
print(len(idealweatherDF))
idealweatherDF.head()

5


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
293,293,belmonte,-15.8631,-38.8828,74.35,76,0,2.86,BR,1716857279
436,436,sirjan,29.4520,55.6814,73.51,15,0,2.35,IR,1716857448
508,508,thomson,33.4707,-82.5046,80.22,89,0,3.44,US,1716857534
546,546,hassi messaoud,31.6804,6.0729,76.86,22,0,3.44,DZ,1716857578
563,563,lingyuan,41.2400,119.4011,71.94,26,0,4.07,CN,1716857597


### Step 3: Create a new DataFrame called `hotel_df`.

In [11]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = idealweatherDF[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
293,belmonte,BR,-15.8631,-38.8828,76,
436,sirjan,IR,29.4520,55.6814,15,
508,thomson,US,33.4707,-82.5046,89,
546,hassi messaoud,DZ,31.6804,6.0729,22,
563,lingyuan,CN,41.2400,119.4011,26,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [15]:
# Set parameters to search for a hotel
radius = 10000
params = {
        "categories" : "accomodation.hotel",
    "apiKey" : geoapify_key,
    "limit" : 1
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude}, {latitude}, {radius}"
    params["bias"] = f"proximity: {longitude}, {latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
belmonte - nearest hotel: No hotel found
sirjan - nearest hotel: No hotel found
thomson - nearest hotel: No hotel found
hassi messaoud - nearest hotel: No hotel found
lingyuan - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
293,belmonte,BR,-15.8631,-38.8828,76,No hotel found
436,sirjan,IR,29.4520,55.6814,15,No hotel found
508,thomson,US,33.4707,-82.5046,89,No hotel found
546,hassi messaoud,DZ,31.6804,6.0729,22,No hotel found
563,lingyuan,CN,41.2400,119.4011,26,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE

# Display the map
# YOUR CODE HERE

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)